In [21]:
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Markdown, display
from datetime import datetime as dt
from ipywidgets import interact
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
import cufflinks
from plotly.subplots import make_subplots
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

In [99]:
x1 = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
x2 = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'
x3 = 'https://raw.githubusercontent.com/abhim-12/Covid-19/master/population.csv'
x4 = 'https://raw.githubusercontent.com/abhim-12/Covid-19/master/state-codes.csv'

In [23]:
temp = []
temp.append(x1.split('_')[-2].title())
temp.append(x2.split('_')[-2].title())

In [100]:
conf = pd.read_csv(x1)
death = pd.read_csv(x2)
pop = pd.read_csv(x3)
codes = pd.read_csv(x4)

In [25]:
l= []
l.append(conf)
l.append(death)

<h1> <center> <a id =6> </a> <span style='font-family:Helvetica'><font color='red'>Covid-19</font> Dashboard </span>

<h3><p>Contents:</p>

1. [Raw Data](#1) <br>
2. [Impact on the Country](#2)<br>
3. [Timeline](#3)<br> 
4. [States affected](#4)<br>
5. [Consolidated State Data](#5)<br>
a. [CFR and Infected](#7)<br>
b. [Percentage Affected](#8)<br>
6. [Consolidated State Specific Data](#9)<br>
7. [Consolidated Province Specific Data](#10)<br>
8. [Weekly Data](#11)<br>
a. [Day-of-the-week Data](#12)<br>

<h3> <a id=1></a>What the raw data looks like :

In [26]:
for i,j in zip(l,temp):
    display(Markdown("**<center>{}**".format(j)))
    display(i.head())
    #print(i.shape)
    print()

**<center>Confirmed**

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,77,82,84,93,112,113,121,121,128,130
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,6,6,6,6,6,6,6,6,6,6
3,630,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.2208,-66.5901,...,286,316,316,452,475,513,573,620,683,725
4,850,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.3358,-64.8963,...,30,30,37,40,42,43,43,45,45,50


**<center>Deaths**

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,3,3,4,4,4,4,4,4,4,4
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,1,1,1,1,1,1,1,1,1,1
3,630,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.2208,-66.5901,...,11,12,15,18,20,21,23,24,33,39
4,850,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.3358,-64.8963,...,0,0,0,0,1,1,1,1,1,1


In [27]:
del death['Population']
def filt(df):
    df.drop(columns=['iso3','code3','Admin2','Country_Region'],axis=1,inplace=True)
    df.rename(columns={'Province_State':'State'},inplace=True)
    df=df.reset_index()
    del df['index']
    #return df

In [28]:
filt(conf)
filt(death)

In [29]:
#Declare all data to be used

In [102]:
def highlight_col(x):
    if x['Case Fatality Rate']>z2['Case Fatality Rate'].mean():
        return ['background-color: lightcoral']*7
    else:
        return ['background-color: gray']*7

In [31]:
def nulls(df):                                    # Check for nulls
    return df.isna().sum()

In [32]:
z = pd.merge(conf,death,on='UID',how='inner')
z.drop(columns=['iso2_y','FIPS_y','State_y','Lat_y','Long__y','Combined_Key_y'],axis=1,inplace=True)
z.rename(columns={'iso2_x':'iso2','FIPS_x':'FIPS','State_x':'State','Lat_x':'Lat','Long__x':'Long',
                 'Combined_Key_x':'Combined_Key'},inplace=True)

In [33]:
nulls(z)                                         # Check for nulls
z.loc[z['FIPS'].isna(),'FIPS']=0

In [34]:
confirm =''                             # Get the latest date of data available
deaths = ''
for i in z.columns.to_list():
    if(i.rfind('x')>0):
        confirm=i
    if(i.rfind('y')>0):
        deaths=i

In [35]:
temp2 = pd.DataFrame()                                # intialize the population table
temp2 = pop[['State','Population']]

In [36]:
cols = conf.columns.to_list()[-1]
ctemp = conf.groupby('State').sum()[conf.columns[-1]].sort_values(ascending=False).reset_index()
dtemp = death.groupby('State').sum()[death.columns[-1]].sort_values(ascending=False).reset_index()
ctemp = ctemp[['State',conf.columns[-1]]]
dtemp = dtemp[['State',death.columns[-1]]]

z2 = pd.merge(ctemp,dtemp,on=['State'],how='inner').merge(temp2,on='State',how='left')      # State level aggregated data
z2.rename(columns={deaths:'Deaths',confirm:'Cases'},inplace=True)

z2['CMR'] = (z2['Deaths']/z2['Population'])*100000                          # Crude Mortality Rate per 100000
z2['Infected %'] = (z2['Cases']/z2['Population'])*100                       # Percentage of population infected

z2['Case Fatality Rate'] = (z2['Deaths']/z2['Cases'])*100                   # Case Fatality Rate - Ratio of Deaths out of those infected
z2 = z2[['State','Cases','Deaths','Population','Infected %','CMR','Case Fatality Rate']]

In [37]:
t2 = pd.merge(z2,codes,on='State',how='left')           # To plot region-wise data

In [38]:
l2=z2.copy(deep=True)                       #Grouping data from z2 for Pie Chart representation
l2= l2.reset_index()
l2.loc[l2['index']>15,'State']='Others'
l2 = l2.groupby('State').sum().reset_index()
l2 = l2.sort_values('Cases',ascending=False).reset_index(drop=True)
l2.drop(columns=['index'],inplace=True)

In [95]:
st = l2.loc[l2['Cases']==l2['Cases'].max()]['State'][0]                     # Explaining the Pie Chart
pcent = round((l2.loc[0]['Cases']/l2['Cases'].sum())*100,2)

st2 = l2.loc[l2['Deaths']==l2['Deaths'].max()]['State'][0]
pcent2 = round((((l2[l2['State']==st2]['Deaths'][0])/l2['Deaths'].sum())*100),2)

In [39]:
def transform(df,s):                          # To dcast the data and get it at a state-date level 
    temp = df.copy(deep=True)
    temp.drop(columns=['UID','iso2','Lat','Long_','FIPS'],inplace=True)
    temp = temp.melt(id_vars=['State','Combined_Key'],var_name='Date',value_name=s)
                                              # Source data is cumulative, this gets the numbers per day
    temp2 = temp.groupby(['Combined_Key'])[s].diff().fillna(0).reset_index()
    p = 'PerDay'+'_'+s
    temp2.rename(columns={s:p},inplace=True) 
    temp2 = pd.concat([temp,temp2],axis=1)
    del temp2['index']
    temp2 = temp2.sort_values(['Combined_Key','Date',s,p]).reset_index(drop=True)
    return temp2

In [40]:
conf2 = transform(conf,'Confirm')             # Merge the 2 datasets after calling the function 
death2 = transform(death,'Deaths')
zz = pd.concat([death2,conf2[['Confirm','PerDay_Confirm']]],axis=1)
#zz.head()

In [41]:
cc = zz.copy(deep=True)
cc['Date'] = cc['Date'].apply(lambda x:pd.to_datetime(x))             # Change type to datetime

In [42]:
cc['week'] = cc['Date'].apply(lambda x:x.week)                        # Get week number
cc['weekday'] = cc['Date'].apply(lambda x : x.day_name())             # Get the Day of the week

<h3> <center> Total Infected in the country 

In [43]:
display(Markdown('<h4>Total Confirmed Cases   :      <font color="red">{}</font>'.format(conf[conf.columns[-1]].sum())))
display(Markdown('<h4>Total Deaths :           <font color="red">{}</font>'.format(death[death.columns[-1]].sum())))

<h4>Total Confirmed Cases   :      <font color="red">496530</font>

<h4>Total Deaths :           <font color="red">18581</font>

<h3> <center> <a id=2></a>Impact of <font color='red'> Covid-19</font> across the country

In [44]:
def country(Condition):                         # Geo Plot of Cases/Deaths across the country
    if Condition=='Cases':
        title='Cases across the country'
        x='Cases'
    if Condition=='Deaths':
        title='Deaths across the country'
        x='Deaths'
    data = dict(type='choropleth',
               locations=t2['Code'],
               locationmode='USA-states',
               colorscale='Portland',
               text=t2['State'],
               z=t2[Condition],
               colorbar = {'title':x})
    layout = dict(title = title,
                  geo = dict(scope='usa')
                 )
    choromap = go.Figure(data = [data],layout = layout)
    iplot(choromap)
interact(country,Condition=['Deaths','Cases'])

interactive(children=(Dropdown(description='Condition', options=('Deaths', 'Cases'), value='Deaths'), Output()…

<function __main__.country(Condition)>

<h3> <center> <a id=3></a>Timeline of Impact

In [45]:
def timeline(Condition):
    tempd = cc.groupby('Date').sum().reset_index()
    temph = tempd.loc[tempd['PerDay_Confirm']>0].reset_index(drop=True)
    if Condition=='Deaths':
        title = 'Timeline of Deaths'
        color='red'
    if Condition=='Cases':
        Condition='Confirm'
        title = 'Timeline of Cases'
        color='orange'
    temph.iplot(x='Date',y=Condition,xTitle='Dates',yTitle='Count',title=title,mode='lines+markers',color=color)
interact(timeline, Condition=['Deaths','Cases'])

interactive(children=(Dropdown(description='Condition', options=('Deaths', 'Cases'), value='Deaths'), Output()…

<function __main__.timeline(Condition)>

<h3> <center> <a id=4></a>Top States most affected by <font color='red'>Covid-19</font>

In [62]:
def scplot(n,Condition):
    if Condition=='Deaths':
        color ='red'
        title = 'Top ' + str(n) + ' States with most Deaths'
    else:
        color ='orange'
        title = 'Top ' + str(n) + ' States with most Cases'
    fig = z2.head(n).sort_values(Condition,ascending=False).iplot(kind='bar',x='State',y=Condition,
    title=title,
    xTitle="States",
    yTitle=Condition,
    mode = 'markers',
    color=color)
interact(scplot, n=(5,30,5),Condition=['Deaths','Cases'])

interactive(children=(IntSlider(value=15, description='n', max=30, min=5, step=5), Dropdown(description='Condi…

<function __main__.scplot(n, Condition)>

<b> <u> <center> <a id=5> </a><h3> States with Case Fatality greater than the average are highlighted to show high risk.

<b> What is Crude Mortality Rate (CMR)? </b>
> CRUDE DEATH RATE is the total number of deaths in a specified geographic area divided by the total population for the same geographic area (for a specified time period, usually a calendar year) and multiplied by 100,000.

In [66]:
z2.sort_values('State').reset_index(drop=True).style.apply(highlight_col,axis=1)

,State,Cases,Deaths,Population,Infected %,CMR,Case Fatality Rate
0,Alabama,2947,80,4.90862e+06,0.0600372,1.62979,2.71463
1,Alaska,246,7,734002,0.0335149,0.953676,2.84553
2,American Samoa,0,0,55312,0,0,nan
3,Arizona,3112,97,7.37849e+06,0.0421766,1.31463,3.11697
4,Arkansas,1171,21,3.039e+06,0.0385324,0.691017,1.79334
5,California,21081,583,3.99375e+07,0.052785,1.45978,2.76552
6,Colorado,6202,226,5.84553e+06,0.106098,3.8662,3.64399
7,Connecticut,10538,448,3.56308e+06,0.295756,12.5734,4.25128
8,Delaware,1326,32,982895,0.134908,3.25569,2.41327
9,Diamond Princess,49,0,nan,nan,nan,0


<h3> <center> <a id=7></a><font color='red'>CFR</font> and <font color='orange'>Infected %</font> in various States

<b>What is Case Fatality Rate?</b>
> The proportion of deaths from a certain disease compared to the total number of people diagnosed with the disease.<br>
> CFR = Deaths/Cases <br>

<b> Infected % : </b>
> Percentage of Population infected with Disease<br>
> Infected% = Cases/Population

In [63]:
def line_plot(n,Condition):
    if Condition=='Case Fatality Rate':
        color='red'
        title='CFR in top ' + str(n) + ' States'
    if Condition=='Infected %':
        color='orange'
        title='Infected % in top '+ str(n) + ' States'
    z2.head(n).iplot(x='State',y=Condition,color=color,xTitle='States',
                 yTitle='Rate',title=title,mode='lines+markers')
interact(line_plot,n=(5,50,5),Condition=['Case Fatality Rate','Infected %'])

interactive(children=(IntSlider(value=25, description='n', max=50, min=5, step=5), Dropdown(description='Condi…

<function __main__.line_plot(n, Condition)>

<h3> <a id=8></a><center> Percent <font color='orange'>Cases</font> & <font color='red'> Deaths </font> in various States

In [50]:
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=l2['State'], values=l2['Cases'], name="Cases"),
              1, 1)
fig.add_trace(go.Pie(labels=l2['State'], values=l2['Deaths'], name="Deaths"),
              1, 2)
fig.update_traces(hole=.4, hoverinfo="label+value+name")

fig.update_layout(
    #title_text="Percent Cases & Deaths in various States",
    annotations=[dict(text='Cases', x=0.18, y=0.5, font_size=20, showarrow=False),
                 dict(text='Deaths', x=0.80, y=0.5, font_size=20, showarrow=False)])
f = go.FigureWidget(fig)
f

FigureWidget({
    'data': [{'domain': {'x': [0.0, 0.45], 'y': [0.0, 1.0]},
              'hole': 0.4,
       …

In [107]:
display(Markdown('<center> <b>{}</b> accounts for nearly <b>{}%</b> of all Cases'.format(st,pcent)))
display(Markdown('<center> <b>{}</b> accounts for nearly <b>{}%</b> of all Deaths'.format(st2,pcent2)))

<center> <b>New York</b> accounts for nearly <b>34.71%</b> of all Cases

<center> <b>New York</b> accounts for nearly <b>42.34%</b> of all Deaths

<h3>Most in a Day:

In [51]:
tt = zz.groupby(['Date'])['PerDay_Confirm'].sum().reset_index()
tt = tt.loc[tt['PerDay_Confirm']==tt['PerDay_Confirm'].max(),['Date','PerDay_Confirm']].reset_index(drop=True)
s = 'Most Cases on a Single Day is:  **<font color="red">{}</font>** on  **{}**'.format(tt['PerDay_Confirm'][0],tt['Date'][0])
display(Markdown(s))
tt = zz.groupby(['Date'])['PerDay_Deaths'].sum().reset_index()
tt = tt.loc[tt['PerDay_Deaths']==tt['PerDay_Deaths'].max(),['Date','PerDay_Deaths']].reset_index(drop=True)
s = 'Most Deaths on a Single Day is **<font color="red">{}</font>** on **{}**<br>'.format(tt['PerDay_Deaths'][0],tt['Date'][0])
display(Markdown(s))

Most Cases on a Single Day is:  **<font color="red">35098.0</font>** on  **4/10/20**

Most Deaths on a Single Day is **<font color="red">2104.0</font>** on **4/10/20**<br>

<h3> Most in a State:

In [52]:
ff = zz.groupby('State').max().reset_index()
ff = ff.loc[ff['PerDay_Confirm']==ff['PerDay_Confirm'].max(),['State','PerDay_Confirm']].reset_index(drop=True)
s = 'Most Cases on a day from a State is:  **<font color="red">{}</font>** from  **<u>{}</u>**<br>'.format(ff['PerDay_Confirm'][0],ff['State'][0])
display(Markdown(s))
ff = zz.groupby('State').max().reset_index()
ff = ff.loc[ff['PerDay_Deaths']==ff['PerDay_Deaths'].max(),['State','PerDay_Deaths']].reset_index(drop=True)
s = 'Most Deaths on a day from a State is:  **<font color="red">{}</font>** from  **<u>{}</u>**<br>'.format(ff['PerDay_Deaths'][0],ff['State'][0])
display(Markdown(s))

Most Cases on a day from a State is:  **<font color="red">6147.0</font>** from  **<u>New York</u>**<br>

Most Deaths on a day from a State is:  **<font color="red">670.0</font>** from  **<u>New York</u>**<br>

<h3> <center> <a id=9></a><font color='orange'>Cases</font> & <font color='red'> Deaths </font> from all provinces in a State

<h5> <u> <center> Note: The numbers are cumulative upto the required date</u> 

In [53]:
dates = zz['Date'].unique()
def plots(state,date,Condition):                 # Plot Cases/Deaths aggregated to a Date for a State
    if Condition=='Deaths':
        color='red'
    else:
        color='orange'
    xx = z[z['State']==state]
    xx['Province'] = [s.split(',')[0] for s in  xx['Combined_Key']]
    if Condition.startswith('C'):
        date2 = date + '_x'
    if Condition.startswith('D'):
        date2 = date + '_y'
    xx = xx.sort_values(date2,ascending=False).head(20)
    xx.iplot(kind='bar',x='Province',y=date2, title=(state+' '+Condition+' as on '+ date), xTitle = 'State', yTitle='Count',color=color)
interact(plots,state=z2['State'].sort_values(),date=dates,Condition=['Cases','Deaths'])

interactive(children=(Dropdown(description='state', options=('Alabama', 'Alaska', 'American Samoa', 'Arizona',…

<function __main__.plots(state, date, Condition)>

<h3> <center> <a id=10></a><font color='orange'>Cases</font> & <font color='red'>Deaths</font> each day in a Province

In [64]:
zz['Province'] = [s.split(',')[0] for s in  zz['Combined_Key']]
province = zz['Province'].unique() 
def plots2(State,Province,Condition):                     # Plots Cases/Deaths for a Province per Day
    a = zz.loc[(zz['Province']==Province) & (zz['State']==State) ]
    if Condition.startswith('C'):
        Condition = 'PerDay_Confirm'
        color = 'orange'
    if Condition.startswith('D'):
        Condition = 'PerDay_Deaths'
        color = 'red'
    a = a[a[Condition].gt(0)]
    a = a.sort_values('Date',ascending=True)
    a.iplot(x='Date',y=Condition,kind='bar',xTitle='Dates',yTitle=Condition.split('_')[1],title=(Condition.split('_')[1] + " in " + Province + ", " + State),color=color)
interact(plots2,State=z2['State'].sort_values(),Province = province,Condition=['Cases','Deaths'])

interactive(children=(Dropdown(description='State', options=('Alabama', 'Alaska', 'American Samoa', 'Arizona',…

<function __main__.plots2(State, Province, Condition)>

In [55]:
tempw = cc.groupby('week').sum().reset_index()         # To get the numbers aggregated per week
tempw['no'] = [i for i in range(1,len(tempw)+1)]
days = cc['weekday'].unique()

In [56]:
tempsw = cc.groupby(['State','weekday']).sum().reset_index().sort_values(['State','weekday'])  #Each State-Each Day of the week

<a id=11></a> <h3> <center> <font color='orange'>Cases</font> & <font color='red'>Deaths</font> per Week

In [57]:
def weeks(Condition):
    if Condition=='Deaths':
        y='PerDay_Deaths'
        title='Weekly Count of Deaths'
        color='red'
    if Condition=='Confirm':
        y='PerDay_Confirm'
        title='Weekly Count of Cases'
        color='orange'
    tempw.iplot(kind='bar',x='no',y=y,xTitle='Week',yTitle='Count',title=title,color=color)
interact(weeks,Condition=['Deaths','Confirm'])

interactive(children=(Dropdown(description='Condition', options=('Deaths', 'Confirm'), value='Deaths'), Output…

<function __main__.weeks(Condition)>

<h3> <center> <a id=12> </a> <font color='orange'>Cases</font> & <font color='red'>Deaths</font> each day of the Week

In [58]:
def weeks2(State,Condition):
    temp = tempsw[tempsw['State']==State]
    if Condition=='Deaths':
        y='PerDay_Deaths'
        title='Deaths each day of the Week'
        color='red'
    if Condition=='Confirm':
        y='PerDay_Confirm'
        title='Cases each day of the Week'
        color='orange'
    temp.iplot(kind='bar',x='weekday',y=y,xTitle='Day',yTitle='Count',title=title,color=color)
interact(weeks2,State=z2['State'].sort_values(),Condition=['Deaths','Confirm'])

interactive(children=(Dropdown(description='State', options=('Alabama', 'Alaska', 'American Samoa', 'Arizona',…

<function __main__.weeks2(State, Condition)>

<h3> Sources:

1.Johns Hopkins Dataset from github<br>
2.US State Codes from Kaggle<br>
3.Population from World Population Review<br>

[<p style='text-align:right'> Back To Top </p>](#6)  

<h4> <center> <u> Created By:</u>   Abhishek Mishra

[<p style='text-align:center'>My GitHub</p> ](https://github.com/abhim-12) 
[<p style='text-align:center'>My LinkedIn</p> ](https://www.linkedin.com/in/abhishek-mishra12/) 